In [261]:
from tensorflow import keras

In [262]:
import matplotlib.pyplot as plt

In [263]:
import numpy as np

In [264]:
import pandas as pd

In [265]:
( train_x , train_y ) , ( test_x , test_y ) = keras.datasets.mnist.load_data()

In [266]:
train_x.shape

(60000, 28, 28)

In [267]:
train_x = np.concatenate([ train_x  for i in range(10)])

In [268]:
train_x.shape

(600000, 28, 28)

In [269]:
train_y.shape

(60000,)

In [270]:
test_x.shape

(10000, 28, 28)

In [271]:
test_y.shape

(10000,)

In [272]:
height = 28
width = 28
channels = 1
latent_space = 10

In [273]:
discriminator = keras.Sequential([
    keras.layers.Input(shape=(height,width,channels)),
    keras.layers.Conv2D(filters=128 , kernel_size=(2,2)  , activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=64 , kernel_size=(2,2)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

In [274]:
discriminator.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 27, 27, 128)       640       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 12, 12, 64)        32832     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_44 (Dense)             (None, 100)               230500    
_________________________________________________________________
dense_45 (Dense)             (None, 1)               

In [275]:
discriminator.compile(optimizer='adam',loss='binary_crossentropy')

In [276]:
generator = keras.Sequential([
    keras.layers.Dense(128 * 7 * 7 , input_shape=(10,)),
    keras.layers.Reshape((7,7,128)),
    keras.layers.Conv2DTranspose(64 , kernel_size=(2,2) , strides =2 , padding='same' ),
    keras.layers.LeakyReLU(),
    keras.layers.Conv2DTranspose(64 , kernel_size=(2,2) , strides= 2 , padding='same'),
    keras.layers.LeakyReLU(),    
    keras.layers.Conv2D(channels , kernel_size=(2,2) , padding='same')
    
])

In [277]:
generator.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 6272)              68992     
_________________________________________________________________
reshape_21 (Reshape)         (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_48 (Conv2DT (None, 14, 14, 64)        32832     
_________________________________________________________________
leaky_re_lu_49 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_transpose_49 (Conv2DT (None, 28, 28, 64)        16448     
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 28, 28, 1)       

In [278]:
gan = keras.Sequential([
    generator,
    discriminator
])

In [279]:
gan.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_41 (Sequential)   (None, 28, 28, 1)         118529    
_________________________________________________________________
sequential_40 (Sequential)   (None, 1)                 264073    
Total params: 382,602
Trainable params: 382,602
Non-trainable params: 0
_________________________________________________________________


In [280]:
discriminator.trainable = False

In [281]:
gan.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_41 (Sequential)   (None, 28, 28, 1)         118529    
_________________________________________________________________
sequential_40 (Sequential)   (None, 1)                 264073    
Total params: 382,602
Trainable params: 118,529
Non-trainable params: 264,073
_________________________________________________________________


In [282]:
gan.compile(optimizer='adam',loss='binary_crossentropy')

In [283]:
discriminator.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 27, 27, 128)       640       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 13, 13, 128)       0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 12, 12, 64)        32832     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 2304)              0         
_________________________________________________________________
dense_44 (Dense)             (None, 100)               230500    
_________________________________________________________________
dense_45 (Dense)             (None, 1)               

In [284]:
iterations = 10000
batch_size = 20

In [285]:
import cv2

In [286]:
start = 0
for i in range(iterations):
    
    stop = start + batch_size
    
    random_latent_space = np.random.normal(size=(batch_size,latent_space))
    
    fake_images = generator.predict(random_latent_space)
    
    real_images = train_x[start:stop].reshape((-1,28,28,1))
    
    
    combined_images = np.concatenate([fake_images , real_images])

    
    real_labels = np.concatenate([ np.zeros((batch_size,1)) , np.ones((batch_size,1))])
    
    
    fake_labels = np.ones((batch_size,1))
    
    discriminator.trainable = True
    
    d_loss = discriminator.train_on_batch(combined_images , real_labels)
    
    discriminator.trainable = False   

    
    a_loss = gan.train_on_batch( random_latent_space , fake_labels)
        
    start += batch_size
    
    if i % 100 == 0:
        
        gan.save_weights('gan.h5')
        
        print("discriminator loss" , d_loss )
        print("gan network loss" , a_loss)
        
        img = real_images[0]
        
        cv2.imwrite('MODELS/real_' + str(i) + '.png' , img)
        
        
        img = fake_images[0]
        
        cv2.imwrite('MODELS/fake_' + str(i) + '.png' , img)
        
    
    

discriminator loss 0.34859079122543335
gan network loss 0.7055771946907043
discriminator loss 0.020203057676553726
gan network loss 15.497807502746582
discriminator loss 0.0015364865539595485
gan network loss 11.842446327209473
discriminator loss 5.931102714384906e-05
gan network loss 13.131963729858398
discriminator loss 0.0007707127369940281
gan network loss 11.959858894348145
discriminator loss 0.009294599294662476
gan network loss 10.052716255187988
discriminator loss 0.009852299466729164
gan network loss 16.8698787689209
discriminator loss 1.129100146712858e-09
gan network loss 26.32010841369629
discriminator loss 0.00033134082332253456
gan network loss 20.12612533569336
discriminator loss 0.00030147837242111564
gan network loss 15.493242263793945
discriminator loss 0.02193342335522175
gan network loss 13.002909660339355
discriminator loss 0.05412909388542175
gan network loss 11.102056503295898
discriminator loss 0.00023519578098785132
gan network loss 16.934263229370117
discrimin

KeyboardInterrupt: 